In [ ]:
# Importing required libraries and modules
import logging
import os
import tensorflow as tf
import IPython.display as ipd

os.environ["CUDA_VISIBLE_DEVICES"] = ""
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s (%(module)s:%(lineno)d) %(levelname)s: %(message)s",
)

# Initialize a TTS processor and save it to a different location
processor = AutoProcessor.from_pretrained("./TensorFlowTTS/dump/ljspeech_mapper.json")
processor.save_pretrained("./test_saved")
processor = AutoProcessor.from_pretrained("./test_saved/processor.json")

# Initialize a TTS model (Tacotron2) 
config = AutoConfig.from_pretrained("./TensorFlowTTS/examples/tacotron2/conf/tacotron2.v1.yaml")
tacotron2 = TFAutoModel.from_pretrained(pretrained_path=None, config=config)


config.save_pretrained("./test_saved")
tacotron2.save_pretrained("./test_saved")

#loading pretrained configuration and model
config = AutoConfig.from_pretrained("./Trained Model/config.yml")
tacotron2 = TFAutoModel.from_pretrained("./Trained Model/checkpoints/model-50000.h5", config=config)

# Initialize a vocoder model (MelGAN) using a pretrained model
melgan = TFAutoModel.from_pretrained("tensorspeech/tts-melgan-ljspeech-en", name="melgan")

# Initialize a TTS processor for text-to-speech conversion
processor = AutoProcessor.from_pretrained("tensorspeech/tts-tacotron2-ljspeech-en")

# Define a function to convert text to speech
def text2speech(input_text, text2mel_model, vocoder_model):
    # Convert input text to sequence of phonemes
    input_ids = processor.text_to_sequence(input_text)

    # Generate mel spectrograms, stop token predictions, and alignment history using the text2mel model
    _, mel_outputs, stop_token_prediction, alignment_history = text2mel_model.inference(
        tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        tf.convert_to_tensor([len(input_ids)], tf.int32),
        tf.convert_to_tensor([0], dtype=tf.int32)
    )

    # Generate audio using the vocoder model from mel spectrograms
    audio = vocoder_model(mel_outputs)[0, :, 0]
    
    return mel_outputs.numpy(), alignment_history.numpy(), audio.numpy()

# Example input text in Persian
story = "سلام بر شما"

# Use the text2speech function to generate mel spectrograms and audio
mels, alignment_history, audios = text2speech(story, tacotron2, melgan)

# Play the generated audio with a specified sampling rate
ipd.Audio(audios, rate=22050)
